In [88]:
import camelot
data = camelot.read_pdf('bulletin.pdf', pages='140-147,161-200', flavor='lattice')

In [105]:
def find_cdcs(s):
    s = ''.join(i for i in s if i!='\n')
    s = s.split()
    ans = list()
    for i in range(1,len(s)):
        if s[i-1].isupper() and s[i][0]=='F' and s[i][1:].isdigit() :
            ans.append(s[i-1] + ' ' + s[i])
    return ans
branchcode = {
    "Biological":'B1',
    "Chemistry":'B2',
    "Economics":'B3',
    "Mathematics":'B4',
    "Physics":'B5',
    "Chemical":'A1',
    "Computer":'A7',
    "Electrical":'A3',
    "Communication":'AA',
    "Instrumentation":'A2',
    "Mechanical":'A4'        
}

cdcs = dict()
def make_dict(df):
    bc = ''
    for i in branchcode.keys():
        if (df[0][0].find(i, 50) >= 0):
            bc = bc + branchcode[i]
    
    if (bc=='' or bc[-2]=='B' or bc in cdcs.keys()):
        return
    
    sem = dict()
    roman = {'II':2, 'III':3, 'IV':4, 'V':5}
    for i in df.index:
        if df[0][i] in roman.keys():
            if len(bc)==2:
                sem[2*roman[df[0][i]]-1] = find_cdcs(df[1][i])
                sem[2*roman[df[0][i]]] = find_cdcs(df[3][i])
            else:
                sem[2*roman[df[0][i]]-1] = find_cdcs(df[1][i+1])
                sem[2*roman[df[0][i]]] = find_cdcs(df[3][i+1])
    cdcs[bc] = sem

In [106]:
for i in range(len(data)):
    make_dict(data[i].df)

In [107]:
cdcs

{'A1': {3: ['MATH F211',
   'CHE F211',
   'CHE F214',
   'CHE F213',
   'CHE F212',
   'BITS F225'],
  4: ['ECON F211',
   'MGTS F211',
   'CHE F241',
   'CHE F242',
   'CHE F243',
   'CHE F244'],
  5: ['CHE F312', 'CHE F313', 'CHE F311', 'CHE F314'],
  6: ['CHE F341', 'CHE F342', 'CHE F343'],
  7: [],
  8: ['BITS F412']},
 'A7': {3: ['MATH F211', 'CS F214', 'CS F222', 'CS F213', 'CS F215'],
  4: ['ECON F211', 'MGTS F211', 'CS F211', 'CS F241', 'CS F212', 'BITS F225'],
  5: ['CS F351', 'CS F372', 'CS F301', 'CS F342'],
  6: ['CS F363', 'CS F364', 'CS F303'],
  7: [],
  8: ['BITS F412']},
 'A3': {3: ['MATH F211', 'EEE F211', 'EEE F212', 'EEE F215', 'EEE F214'],
  4: ['ECON F211',
   'MGTS F211',
   'EEE F241',
   'EEE F243',
   'EEE F244',
   'BITS F225'],
  5: ['EEE F311', 'MATH F212', 'ME F344', 'EEE F313'],
  6: ['EEE F341', 'EEE F342', 'EEE F312'],
  7: [],
  8: []},
 'AA': {3: ['MATH F211', 'ECE F211', 'ECE F212', 'ECE F215', 'ECE F214'],
  4: ['ECON F211',
   'MGTS F211',
   'ECE

In [108]:
import json
with open("cdcdict.json", 'w') as f :
    json.dump(cdcs, f)